In [105]:
import re
import spacy
from spacy.matcher import Matcher
import fitz
import os


In [106]:
# Descargar el modelo si no está instalado
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

matcher = Matcher(nlp.vocab)
    

In [107]:
def extraer_metadatos_gaceta(ruta_pdf):
    """
    Extrae metadatos de una     gaceta del Congreso usando un enfoque híbrido.
    """
    metadatos = {}

    try:
        with fitz.open(ruta_pdf) as doc:  
            texto_completo = ""
            for pagina in doc:
                texto_completo += pagina.get_text()
                
            texto_completo = preprocesar_texto(texto_completo)
            doc_spacy = nlp(texto_completo)

            # --- 1. Regex para patrones consistentes ---
            metadatos["numero_gaceta"] = extraer_con_regex(r"GACETA N\.° (\d+/\d+)", texto_completo)
            metadatos["issn"] = extraer_con_regex(r"ISSN (\d{4}-\d{3}[\dX])", texto_completo)
            metadatos["num_paginas"] = extraer_con_regex(r"EDICIÓN DE (\d+) PÁGINAS", texto_completo)

            # 2. Entidad (Senado/Cámara) - Regex (más robusto)
            match_entity = re.search(r"(?i)(SENADO Y CÁMARA|SENADO|CÁMARA)", texto_completo)
            if match_entity:
                entity = match_entity.group(1).strip() 
                if entity == "SENADO Y CÁMARA":
                    metadatos["entidades"] = ["SENADO", "CÁMARA", "SENADO Y CAMARA"]
                else:
                    metadatos["entidades"] = [entity]

            # --- 3. Año (Números Romanos) --- (Regex)
            pattern_anio = r"AÑO\s+([MDCLXVI]+)" #Expresión regular simplificada
            match_anio = re.search(pattern_anio, texto_completo, re.IGNORECASE)

            if match_anio:
                romano = match_anio.group(1)
                metadatos["anio_romano"] = romano
                metadatos["anio"] = romano_a_entero(romano)

            # --- 4. Rama Legislativa --- (Regex)
            pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+PÚBLICO"
            if re.search(pattern_branch, texto_completo):
                metadatos["rama"] = "Rama Legislativa del Poder Público"

             # --- 5. spaCy para entidades y contexto ---

        
            # 5. Directores (Matcher + Regex para mayor robustez)
            patron_directores = [{"LOWER": "directores"}, {"POS": "PROPN", "OP": "+"}]
            matcher.add("DIRECTORES", [patron_directores])
            matches = matcher(doc_spacy)
            directores = []
            for _, start, end in matches:
                span = doc_spacy[start:end]
                texto_directores = span.text
                nombres = re.findall(r"([A-Z]+(?:\s+[A-Z]+)+)", texto_directores)
                directores.extend(nombres)
            metadatos["directores"] = ", ".join(directores)



            # Tipo de Documento / Descripción - Regex + spaCy
            pattern_tipo_documento = r"(?i)(CAMARA DE REPRESENTANTES|PROYECTOS DE LEY|SENADO DE LA REPÚBLICA|COMENTARIOS|PONENCIAS|ACTA|PROYECTO DE LEY|INFORME|RESOLUCIÓN|CONCEPTO|PROPOSICIÓN|CONSTANCIA|OBJECIONES|CONCEPTOS JURÍDICOS|LEYES SANCIONADAS|PRESENTACIÓN)(.*)"
            for oracion in doc_spacy.sents:
                match = re.search(pattern_tipo_documento, oracion.text, re.MULTILINE | re.DOTALL)
                if match:
                    metadatos["tipo_documento"] = match.group(1).strip()
                    metadatos["subtitulo"] = match.group(2).strip()
                    break

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo PDF: {ruta_pdf}")
        return None
    except Exception as e:
        print(f"Error al procesar el PDF {ruta_pdf}: {e}")
        return None

    return metadatos


In [108]:
def extraer_con_regex(patron, texto):

    coincidencia = re.search(patron, texto, re.IGNORECASE)
    if coincidencia:
        return coincidencia.group(1).replace(" ", "") if len(coincidencia.groups()) == 1 else tuple(grupo.replace(" ", "") for grupo in coincidencia.groups())
    return None

In [109]:
def preprocesar_texto(texto):

    texto = texto.replace("-\n", "")
    texto = texto.replace("\n", " ")
    texto = re.sub(r"\s+", " ", texto)
    texto = texto.strip()
    return texto

In [110]:
def romano_a_entero(romano):
    """Convierte un número romano a entero."""
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

In [111]:

# --- Ejemplo de Uso ---
ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64.pdf"
datos_extraidos = extraer_metadatos_gaceta(ruta_gaceta)


if datos_extraidos:
    print(datos_extraidos)

{'numero_gaceta': None, 'issn': None, 'num_paginas': '64', 'entidades': ['SENADO', 'CÁMARA', 'SENADO Y CAMARA'], 'anio_romano': 'XXV', 'anio': 25, 'rama': 'Rama Legislativa del Poder Público', 'directores': '', 'tipo_documento': 'COMENTARIOS', 'subtitulo': 'AL PROYECTO DE LEY NÚMERO 97 DE 2015 SENADO “por la cual se prohíbe la producción, comercialización, exportación, importación y distribución de cualquier variedad de asbesto en Colombia”.'}
